In [4]:
import os
import pandas as pd
from scipy.io import wavfile
import numpy as np
import glob
import re
import math


In [8]:
import thinkdsp

In [6]:
cwd = os.getcwd()
directory = cwd + '/datasets/old_data/hastaverileri/'
wav_files = glob.glob(os.path.join(directory, '**', '*.wav'), recursive=True)

In [10]:
wav = thinkdsp.read_wave(wav_files[0])

In [12]:
thinkdsp.play_wave(filename=wav_files[0], player='aplay')

Playing WAVE '/home/honorsea/Desktop/sentora/heart_diseases/datasets/old_data/hastaverileri/26.wav' : Signed 16 bit Little Endian, Rate 48000 Hz, Mono


In [11]:
data_list = []
patient_list = []

for wav in wav_files:
    patient = int(re.search(r'/(\d+)\.wav$', wav).group(1))
    patient_list.append(patient)
    samplerate, data = wavfile.read(wav)
    data_list.append(data)

data_list

[array([   0,    0,    0, ..., 1449, 1431, 1413], dtype=int16),
 array([   0,    0,    0, ..., -506, -485, -464], dtype=int16),
 array([    0,     0,     0, ..., -2837, -2825, -2809], dtype=int16),
 array([   0,    0,    0, ..., 1759, 1774, 1789], dtype=int16),
 array([    0,     0,     0, ..., -4149, -4110, -4069], dtype=int16),
 array([   0,    0,    0, ..., -150, -149, -148], dtype=int16),
 array([   0,    0,    0, ..., 1143, 1154, 1165], dtype=int16),
 array([ 0,  0,  0, ..., 85, 80, 75], dtype=int16),
 array([ 0,  0,  0, ..., 90, 93, 95], dtype=int16),
 array([ 0,  0,  0, ..., 76, 74, 71], dtype=int16),
 array([   0,    0,    0, ..., -875, -883, -889], dtype=int16),
 array([ 0,  0,  0, ..., 15,  7, -1], dtype=int16),
 array([   0,    0,    0, ..., 1408, 1417, 1425], dtype=int16),
 array([    0,     0,     0, ..., -2482, -2472, -2460], dtype=int16),
 array([   0,    0,    0, ..., -586, -597, -608], dtype=int16),
 array([   0,    0,    0, ..., -476, -438, -399], dtype=int16),
 array

In [15]:
wav_files = sorted(wav_files, key=lambda x: int(x.split('/')[-1].split('.')[0]))

In [16]:
wav_df = pd.DataFrame(columns=['patient', 'rate_min', 'rate_max', 'rate_std', 'rate_avg'])
wav_df

,patient,rate_min,rate_max,rate_std,rate_avg


In [17]:
timeseries_df = pd.read_csv(cwd + '/datasets/timeseriesdata.csv')
timeseries_df = timeseries_df.drop('Unnamed: 0', axis=1)

In [21]:
df_list = list()
for wav in wav_files:
    patient = int(re.search(r'/(\d+)\.wav$', wav).group(1))
    samplerate, data = wavfile.read(wav)
    resampler = math.floor(len(data) / len(timeseries_df[timeseries_df['patient']==patient]))
    del_val = len(data) - (resampler*len(timeseries_df[timeseries_df['patient']==patient]))
    del_val = math.ceil(len(data)/del_val)
    indices_to_delete = np.arange((del_val-1), len(data), del_val)

    modified_array = np.delete(data, indices_to_delete)
    num_chunks = len(modified_array) // resampler
    reshaped_array = modified_array[:num_chunks * resampler].reshape(num_chunks, resampler)
    averages = reshaped_array.mean(axis=1)
    num_chunks
    descriptive_stats = []

    for chunk in reshaped_array:
        stats = {
            'patient':patient, 
            'rate_min': np.min(chunk),
            'rate_max': np.max(chunk),
            'rate_std': np.std(chunk),
            'rate_avg': np.mean(chunk)
        }
        descriptive_stats.append(stats)
    df_list.append(pd.DataFrame(descriptive_stats))

wav_df = pd.concat(df_list)



In [22]:
wav_df

,patient,rate_min,rate_max,rate_std,rate_avg
0,1,0,0,0.000000,0.000000
1,1,0,0,0.000000,0.000000
2,1,0,0,0.000000,0.000000
3,1,0,0,0.000000,0.000000
4,1,0,0,0.000000,0.000000
...,...,...,...,...,...
4056,41,-1534,911,797.614263,57.552448
4057,41,-1538,2144,1328.394839,550.286713
4058,41,602,1880,257.112438,1184.440559
4059,41,-2660,565,903.267175,-1232.818182


In [23]:

timeseries_df

,Millis,ECG,IR,RED,patient,datetime
0,892,226.0,28556.0,28966.0,1,2023-10-18 09:38:39
1,893,226.0,28554.0,28959.0,1,2023-10-18 09:38:39
2,893,226.0,28556.0,28960.0,1,2023-10-18 09:38:39
3,893,226.0,28552.0,28959.0,1,2023-10-18 09:38:39
4,894,226.0,28551.0,28954.0,1,2023-10-18 09:38:39
...,...,...,...,...,...,...
547272,157,219.0,26252.0,29845.0,41,2023-10-27 15:56:43
547273,168,219.0,26254.0,29843.0,41,2023-10-27 15:56:43
547274,170,219.0,26252.0,29844.0,41,2023-10-27 15:56:43
547275,172,219.0,26250.0,29843.0,41,2023-10-27 15:56:43


In [24]:
wav_df.to_csv('datasets/wav_encoding.csv')